In [4]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter1.util.importer_GRU import *
MODEL_PATH = 'model_saves/ex10_model'
uk_path = "../../src/data_preprocess/dataset/UKDATA_cleaned.csv"

In [5]:
assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = GRU(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .set_inference_samples(inference_samples) \
    .set_inference_dropout(inference_dropout) \
    .add_test_error(NRMSE) \
    .add_test_error(NMLSCV) \
    .add_test_error(NMCRPS) \
    .build()

model.fit()
print("VALIDATION LOSS GRAPH")
plot_loss(model.get_validation_loss())
print("TRAINING LOSS GRAPH")
plot_loss(model.get_training_loss())
torch.save(model.state_dict(), MODEL_PATH + ".pth")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

LR finder stopped early after 99 steps due to diverging loss.
Learning rate set to 0.002754228703338169
Restoring states from the checkpoint path at /home/scoop/experimentation/experiments/iter1/.lr_find_aadbd542-4efd-474b-80e8-c56131aa84c5.ckpt
Restored all states from the checkpoint at /home/scoop/experimentation/experiments/iter1/.lr_find_aadbd542-4efd-474b-80e8-c56131aa84c5.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 2 succeeded, trying batch size 4
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 8 succeeded, trying batch size 16
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 16 succeeded, trying batch size 32
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 32 succeeded, trying batch size 64
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 64 succeeded, trying batch size 128
`Trainer.fit` stopped: `

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

VALIDATION LOSS GRAPH


TRAINING LOSS GRAPH


In [6]:

from experiments.iter1.util.evaluate import evaluate_model

df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = GRU(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .set_inference_samples(inference_samples) \
    .set_inference_dropout(inference_dropout) \
    .add_test_error(NRMSE) \
    .add_test_error(NMLSCV) \
    .add_test_error(NMCRPS) \
    .build()

model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
model.test()

evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


FileNotFoundError: [Errno 2] No such file or directory: 'model_saves/ex10_model'